---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    file = open('university_towns.txt')
    txt = file.read()
    
    txt = txt.split('\n')[:-1]
    university_towns = []
    for line in txt:
        if '[edit]' in line:
            state = line[:-6]
        else:
            university_towns.append(([state, line.split(' (')[0]]))
    return pd.DataFrame(university_towns, columns=['State', 'RegionName'])


get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4,6], index_col=0, names=['Quarter', 'GDP'], header=None)
    df_gdp['Recession'] = False
    
    for i in range(2,len(df_gdp) - 2):
        gdp_2ago_q = df_gdp.iloc[i-2,0]
        gdp_last_q = df_gdp.iloc[i-1,0]
        gdp_this_q = df_gdp.iloc[i,0]
        gdp_next_q = df_gdp.iloc[i+1,0]
        gdp_ = df_gdp.iloc[i+2,0]
        last_quarter_is_recession = df_gdp.iloc[i-1,1]
        
        # Start of recession
        if (not last_quarter_is_recession and (gdp_last_q > gdp_this_q > gdp_next_q)):
            df_gdp.iloc[i,1] = True
        # End of recession
        elif (last_quarter_is_recession and (gdp_this_q > gdp_last_q > gdp_2ago_q)):
            df_gdp.iloc[i,1] = False
        # Middle of recession
        else:
            df_gdp.iloc[i,1] = last_quarter_is_recession

    return df_gdp[df_gdp['Recession'] == True].iloc[0].name

get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4,6], index_col=0, names=['Quarter', 'GDP'], header=None)
    df_gdp['Recession'] = False
    
    for i in range(2, len(df_gdp) - 2):
        gdp_2ago_q = df_gdp.iloc[i-2,0]
        gdp_last_q = df_gdp.iloc[i-1,0]
        gdp_this_q = df_gdp.iloc[i,0]
        gdp_next_q = df_gdp.iloc[i+1,0]
        gdp_ = df_gdp.iloc[i+2,0]
        last_quarter_is_recession = df_gdp.iloc[i-1,1]
        
        # Start of recession
        if (not last_quarter_is_recession and (gdp_last_q > gdp_this_q > gdp_next_q)):
            df_gdp.iloc[i,1] = True
        # End of recession
        elif (last_quarter_is_recession and (gdp_this_q > gdp_last_q > gdp_2ago_q)):
            df_gdp.iloc[i,1] = False
        # Middle of recession
        else:
            df_gdp.iloc[i,1] = last_quarter_is_recession
    for i in range(1, len(df_gdp)):
        if (not df_gdp.iloc[i,1] and df_gdp.iloc[i-1,1]):
            return df_gdp.iloc[i].name
    return

get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4,6], index_col=0, names=['Quarter', 'GDP'], header=None)
    df_gdp['Recession'] = False
    
    for i in range(2, len(df_gdp) - 2):
        gdp_2ago_q = df_gdp.iloc[i-2,0]
        gdp_last_q = df_gdp.iloc[i-1,0]
        gdp_this_q = df_gdp.iloc[i,0]
        gdp_next_q = df_gdp.iloc[i+1,0]
        gdp_ = df_gdp.iloc[i+2,0]
        last_quarter_is_recession = df_gdp.iloc[i-1,1]
        
        # Start of recession
        if (not last_quarter_is_recession and (gdp_last_q > gdp_this_q > gdp_next_q)):
            df_gdp.iloc[i,1] = True
            recession_min = df_gdp.iloc[i,0]
            min_quarter = df_gdp.iloc[i].name
        # End of recession
        elif (last_quarter_is_recession and (gdp_this_q > gdp_last_q > gdp_2ago_q)):
            df_gdp.iloc[i,1] = False
        # No transition
        elif (last_quarter_is_recession):
            df_gdp.iloc[i,1] = last_quarter_is_recession
            if (df_gdp.iloc[i,0] < recession_min):
                recession_min = df_gdp.iloc[i,0]
                min_quarter = df_gdp.iloc[i].name
        else:
            df_gdp.iloc[i,1] = last_quarter_is_recession
            
    return min_quarter


get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    cols_to_drop = ['Metro', 'CountyName', 'SizeRank', 'RegionID','1996-04', 
                      '1996-05', '1996-06', '1996-07', '1996-08', 
                      '1996-09', '1996-10', '1996-11', '1996-12', 
                      '1997-01', '1997-02', '1997-03', '1997-04', 
                      '1997-05', '1997-06', '1997-07', '1997-08', 
                      '1997-09', '1997-10', '1997-11', '1997-12', 
                      '1998-01', '1998-02', '1998-03', '1998-04', 
                      '1998-05', '1998-06', '1998-07', '1998-08', 
                      '1998-09', '1998-10', '1998-11', '1998-12', 
                      '1999-01', '1999-02', '1999-03', '1999-04', 
                      '1999-05', '1999-06', '1999-07', '1999-08', 
                      '1999-09', '1999-10', '1999-11', '1999-12']
    df_housing = df_housing.drop(cols_to_drop, axis=1)
    df_housing['State'] = df_housing['State'].map(states)
    df_housing = df_housing.set_index(['State', 'RegionName'])

    def to_quarter(row, *args):
        prices = []
        for month in args:
            prices.append(row[month])
        return np.mean(prices)

    quarter_format = '%4dq%d'
    date_format = '%4d-%02d'
    for year in range(2000, 2017):
        for quarter in range(1,5):
            if (quarter > 3 and year >= 2016):
                pass
            else:
                opening_month = ((quarter - 1) * 3) + 1
                months = []
                for month in range(opening_month, opening_month + 3):
                    if (year == 2016 and month >= 9):
                        pass
                    else:
                        months.append(date_format % (year, month))
                quarter_name = quarter_format % (year, quarter)
                df_housing[quarter_name] = df_housing.apply(to_quarter, axis=1, args=months)
                df_housing.drop(months, inplace=True, axis=1)
    return df_housing

In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Load university towns dataframe
    university_towns = get_list_of_university_towns()
    # set marker column to identify university towns after merge
    university_towns['University'] = True
    
    # Merge above df with housing quarters
    df_combined = pd.merge(convert_housing_data_to_quarters(), university_towns, left_index=True, right_on=['State', 'RegionName'], how='outer')
    # set non-university towns to False
    df_combined['University'].fillna(False, inplace=True)
    # Set crash = bottom of recession / start of recession (price ratio)
    df_combined['Crash'] = df_combined[get_recession_bottom()] / df_combined[get_recession_start()]
    df_combined.set_index('RegionName', inplace=True)
    # Drop unneeded columns and NA values
    df_combined = (df_combined[['University', 'Crash']].dropna())
    
    # Create 2 series with only price ratios for university
    s_university = df_combined[df_combined['University'] == True]['Crash']
    # Create 2 series with only price ratios for non-university
    s_not_university = df_combined[df_combined['University'] == False]['Crash']
    
    better = "university town" if np.mean(s_university) > np.mean(s_not_university) else "non-university town"
    t_result = ttest_ind(s_university, s_not_university)

    
    
    p_value = t_result.pvalue
    different = "True" if p_value < .01 else False

    
    return (different, p_value, better)

run_ttest()

('True', 0.009243914359628132, 'university town')